In [12]:
import pandas as pd
import numpy as np
import datetime
from scipy.optimize import minimize

In [41]:
data = pd.read_csv('second_df_with_RM_30_5_H2.csv', parse_dates=["time"])
data.head()

,time,close,time.1,RM,r,rv
0,2019-09-30 10:00:00,52.013527,2019-09-30 10:00:00,NaN,NaN,NaN
1,2019-09-30 10:30:00,52.018470,2019-09-30 10:30:00,0.021285,0.004944,0.000024
2,2019-09-30 11:00:00,52.062964,2019-09-30 11:00:00,0.006196,0.044494,0.001980
3,2019-09-30 11:30:00,52.171728,2019-09-30 11:30:00,0.014816,0.108763,0.011829
4,2019-09-30 12:00:00,52.255772,2019-09-30 12:00:00,0.020385,0.084044,0.007063


In [21]:
start = datetime.datetime(2019,9, 30).date()
end = datetime.datetime(2019,9, 30).date()
# end = datetime.datetime(2019,11, 30).date()
df = data[(data['time'].dt.date >= start) & (data['time'].dt.date <= end)]
rv = df['rv'].fillna(0)
RM = df['RM'].fillna(0)

In [38]:
class HEAVY():
    def __init__(self, alpha=0.5, beta=0.3, alphaR=0.5, betaR=0.3):
        self.alpha = alpha
        self.alphaR = alphaR
        self.beta = beta
        self.betaR = betaR
        
    def log_Q1(self, alpha, beta, rv, RM):
        T = len(rv)
        h = T**0.5 * np.sum(rv[:int(T**0.5)])
        ll = 0

        for i in range(1, T):
            h = self.mu*(1 - alpha*self.k - beta) + alpha*RM[i-1] + beta*h
            ll += 0.5*(np.log(h+1e-9) + rv[i]/h)
        return ll

    def log_Q2(self, alphaR, betaR, rv, RM):
        T = len(rv)
        mu_t = T**0.5 * np.sum(RM[:int(T**0.5)])
        ll = 0

        for i in range(1, T):
            mu_t = self.muR*(1 - alphaR - betaR) + alphaR*RM[i-1] + betaR*mu_t
            ll += 0.5*(np.log(mu_t+1e-8) + RM[i]/mu_t)

        return ll

    def fit(self, rv, RM):
        self.mu = rv.mean()
        self.muR = RM.mean()
        self.k = self.mu / self.muR
        
        fun1 = lambda params: self.log_Q1(params[0], params[1], rv, RM)
        cons1 = (
            {'type': 'ineq', 'fun': lambda x:  x[0]}, 
            {'type': 'ineq', 'fun': lambda x:  x[1]},
            {'type': 'ineq', 'fun': lambda x:  1 - x[1]}
        )
        self.alpha, self.beta = minimize(fun1, (self.alpha, self.beta), method='SLSQP', tol=1e-6, constraints=cons1).x
        
        fun2 = lambda params: self.log_Q2(params[0], params[1], rv, RM)
        cons2 = (
            {'type': 'ineq', 'fun': lambda x:  -x[0] - x[1] + 1}, 
            {'type': 'ineq', 'fun': lambda x:  x[0]}, 
            {'type': 'ineq', 'fun': lambda x:  x[1]}
        )
        self.alphaR, self.betaR = minimize(fun2, (self.alphaR, self.betaR), method='SLSQP', tol=1e-6, constraints=cons2).x
        

In [39]:
model = HEAVY()
model.fit(rv, RM)
print(model.alpha, model.beta, model.alphaR, model.betaR)

1.3315233349996902 0.10856719942928156 0.6723906336469317 0.1965071719417836


<ipython-input-38-80022d6f6a31>:15: RuntimeWarning: invalid value encountered in log
  ll += 0.5*(np.log(h+1e-9) + rv[i]/h)
